# 외부데이터  
  
#### 기상청 API_날씨데이터

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import requests
import numpy as np

In [ ]:
## 대표코드
## stnlds(지역) : 대전(133)으로 설정

import json

url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
my_key = '********' # 인증키_str형식

params ={'serviceKey' : my_key, 
         'pageNo' : '1', 
         'numOfRows' : '999', # 1000 이상일 시 응답 에러
         'dataType' : 'JSON', 
         'dataCd' : 'ASOS', 
         'dateCd' : 'DAY', 
         'startDt' : '20170101', # start_date
         'endDt' : '20171231', # end_date
         'stnIds' : '133' }


response = requests.get(url, params=params)
r_dict = json.loads(response.content)
r_response = r_dict.get("response")
r_body = r_response.get("body")
r_items = r_body.get("items")
r_item = r_items.get("item")

city = [] #도시
time = [] #일자
tem = [] #온도
rain = [] #강수여부
hum = [] #습도
snow = [] # 합계3시간 신적설 (3시간 동안 내린 양)


empty_data = pd.DataFrame()

for i in range(len(r_item)):
    time.append(r_item[i]['tm'])
    city.append(r_item[i]['stnNm'])
    tem.append(r_item[i]['avgTa'])
    rain.append(r_item[i]['sumRn'])
    hum.append(r_item[i]['avgRhm'])
    snow.append(r_item[i]['sumDpthFhsc'])

time = pd.Series(time)
city = pd.Series(city)
tem = pd.Series(tem)
rain = pd.Series(rain)
hum = pd.Series(hum)
snow = pd.Series(snow)

data = [ time, city, tem, rain, hum, snow ]

df = pd.concat(data, axis=1)

df.columns = ['일자','시도','기온','강수량','습도','신적설량']
df.replace('', 0,inplace=True)

df['일자'] = pd.to_datetime(df['일자'])
df['기온'] = round(df['기온'].astype('float32'),1)
df['강수량'] = round(df['강수량'].astype('float32'),1)
df['습도'] = round(df['습도'].astype('float32'),1)
df['신적설량'] = round(df['신적설량'].astype('float32'),1)

df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일자      365 non-null    datetime64[ns]
 1   시도      365 non-null    object        
 2   기온      365 non-null    float32       
 3   강수량     365 non-null    float32       
 4   습도      365 non-null    float32       
 5   신적설량    365 non-null    float32       
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 11.5+ KB


,일자,시도,기온,강수량,습도,신적설량
0,2017-01-01,대전,3.5,0.0,81.500000,0.0
1,2017-01-02,대전,4.7,0.0,83.000000,0.0
2,2017-01-03,대전,3.3,0.0,77.099998,0.0
3,2017-01-04,대전,4.2,0.0,70.500000,0.0
4,2017-01-05,대전,4.3,0.0,71.300003,0.0
...,...,...,...,...,...,...
360,2017-12-27,대전,-5.1,0.0,47.400002,0.0
361,2017-12-28,대전,-1.6,0.0,57.900002,0.0
362,2017-12-29,대전,2.9,0.0,78.900002,0.0
363,2017-12-30,대전,3.5,0.9,80.800003,0.0
